# DSGE modeling: solution and simulation

##### This python script tries to replicate the paper of Cooley and Hansen 1989. The main difference with respect to the paper is that I use a different algorithm to solve the model based on Klein 2000. Before running the code, you can fastly read the report on the same github repository, which introduces you to the model and it shows the main equations, the FOCs, the steady state and the log-linearization. Notice that the order of the variable used in the script, it's the same of the one in the report.

In [1]:
# Import libraries. No need for specific versions.
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy
import scipy.stats
import statsmodels.api as sm
from tabulate import tabulate
import math
from tqdm import tqdm

##### The following function is based on the Schur decomposition and the solution algorithm written by Klein (2000)

In [2]:
# POLICY AND TRANSITION FUNCTION - KLEIN(2000)

def compute_policy_transition(A, B, nk, nvar):
    
    #SCHUR DECOMPOSITION
    AA,BB,a1,b1,q1,z1=la.ordqz(A, B, sort='ouc', output='real')
    
    #FILL IN ORDERED GENERALIZED EIGENVALUES
    geEigenvalues=np.zeros(nvar)
    for ii in range(nvar):
        geEigenvalues[ii]=np.real(BB[ii,ii]/AA[ii,ii]) 
    
    #CHECK STABILITY PROPERTIES
    if abs(geEigenvalues[nk-1])<1:
        print ("Generalized decomposition satisfies stability property")
    if abs(geEigenvalues[nk-1])>1:
        print ("Problems with Generalized decomposition ")

    #Z1 IS THE RIGHT SCHUR MATRIX
    #PARTITION z
    z1=np.asmatrix(z1)
    z11 = z1[0:nk,0:nk]  #stable part of z
    z21 = z1[nk:nvar,0:nk]  #unstable part
    z21=np.asmatrix(z21)
    z11=np.asmatrix(z11)

    z12=z1[0:nk,nk:nvar]
    z12=np.asmatrix(z12)
    z22=z1[nk:nvar,nk:nvar]
    z22=np.asmatrix(z22)

    z11i = np.linalg.inv(z11)  #z11i is inverse of z11
    z11i=np.asmatrix(z11i)   

    #CHECK INVERTIBILITY CONDITION
    if np.linalg.matrix_rank(z11)<nk:
    	print ('Invertibility condition violated')
    else:
        print('Unique solution')

    #LET'S COMPUTE THE UPPER ELEMENTS OF THE UPPER TRIANGULAR MATRICES
    a11 = AA[0:nk,0:nk]
    a11 = np.asmatrix(a11)

    b11 = BB[0:nk,0:nk]
    b11 = np.asmatrix(b11)
    
    #POLICY FUNCTION
    G = np.real(-np.linalg.inv(z22.transpose())*z12.transpose())
    
    #TRANSITION FUNCTION
    dyn = np.linalg.inv(a11)*b11
    H = np.real(z11*dyn*z11i)
    
    return(G, H)

##### In the following two blocks we bring the state space model at the end of the report: Aw(t+1)=Bw(t) into python. Then we use the solution alogorithm reported above to extract the policy and transition functions, such that we can simulate the economies and extract the summary statistics.

In [3]:
#LET'S SET UP THE PARAMETERS OF THE MODEL
beta=0.99
delta=0.025
theta=0.36
matrixA=1.72
h0=0.583
matrixB=(matrixA*np.log(1-h0))/h0
gamma=0.95
alpha=0.48

#STATE VALUES
gbar=1.015
rbar=(1/beta)-(1-delta)
wbar=(1-theta)*((rbar/theta)**(theta/(theta-1)))
cbar=-((beta*wbar)/(gbar*matrixB))
pbar=(1/cbar)
kbar=cbar/((rbar/theta)-delta)
hbar=((rbar/theta)**(1/(1-theta)))*kbar
ybar=(kbar**theta)*(hbar**(1-theta))
#ybar=cbar+delta*kbar

#NUMBER OF VARIABLES, AND NUMBER OF STATE VARIABLES
nvar=8
nk=3

In [4]:
#LET'S EXPRESS THE MODEL AS Aw(t+1)=Bw(t) - AFTER COMPUTING THE LOG-LINEARIZATION/LINEARIZATION
A=np.asmatrix(np.zeros((nvar,nvar)))
B=np.asmatrix(np.zeros((nvar,nvar)))

#LET'S COSTRUCT MATRIX A
A[0,3]=1
A[0,4]=-beta*rbar
A[1,2]=1
A[1,5]=1
A[1,6]=1
A[3,0]=kbar
A[6,1]=1
A[7,2]=1
#LET'S CONSTRUCT MATRIX B
B[0,3]=1
B[1,3]=1
B[1,5]=1
B[2,5]=1
B[2,6]=1
B[3,0]=(1-delta+rbar)*kbar
B[3,3]=wbar*hbar
B[3,4]=rbar*kbar
B[3,5]=(1/pbar)
B[3,7]=wbar*hbar
B[4,0]=theta
B[4,1]=1
B[4,3]=-1
B[4,7]=-theta
B[5,0]=theta-1
B[5,1]=1
B[5,4]=-1
B[5,7]=-(theta-1)
B[6,1]=gamma
B[7,2]=alpha

#COMPUTE POLICY AND TRANSITION FUNCTIONS
G, H = compute_policy_transition(A, B, nk, nvar)

print("Transition Function in the Hansen Model")
print (H)
print("Policy Function in the Hansen Model")
print (G)

Generalized decomposition satisfies stability property
Unique solution
Transition Function in the Hansen Model
[[ 9.41816660e-01  1.55228314e-01  2.71253428e-02]
 [ 0.00000000e+00  9.50000000e-01 -5.23798816e-17]
 [ 0.00000000e+00  0.00000000e+00  4.80000000e-01]]
Policy Function in the Hansen Model
[[ 0.53158781  0.4702745   0.03122343]
 [-0.94504499  1.94173422 -0.05550833]
 [-0.53158781 -0.4702745   0.44877657]
 [ 0.53158781  0.4702745  -0.44877657]
 [-0.4766328   1.47145973 -0.08673176]]


C:\Users\matti\AppData\Local\Temp/ipykernel_16660/305111649.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  geEigenvalues[ii]=np.real(BB[ii,ii]/AA[ii,ii])


##### We proceed by simulating the path of the different variables. For each simulation, we produce firstly the state variables (k, z, g) and then we compute the control variables (w, r, p, c, h, y, i). Notice that y (output) and i (investment) were not in the state space model, since they can be retrieved from the values of the other variables, 

In [5]:
#LET'S SETUP THE MONTECARLO PARAMETERS
m=3  #state variables
T=115
mc_reps=1000
var=10 # since we also use investments and output not included in the state space model above
sstd=np.zeros((mc_reps,var))
scorr=np.zeros((mc_reps,var))
sigma1=0.00721 #shock to productivity
sigma2=0 #no shock to money growth
mu1=(math.log(1))*(1-gamma) # productivity shock mean
mu2=(math.log(1.015))*(1-alpha) # money growth shock mean

In [6]:
#LET'S COMPUTE THE MONTECARLO DRAWS - NO SHOCK TO MONEY GROWTH
for i in tqdm(range(mc_reps)):
    eps = np.zeros((T,m))  #where we magazine the state variables
    shock1 = np.random.normal(mu1,sigma1,T)
    shock2 = np.random.normal(mu2,sigma2,T)
    eps[:,1]=shock1
    eps[:,2]=shock2
    #GENERATE THE STATE VARIABLES
    s=np.zeros((T,m))
    sc=np.zeros((1,3)) #temporary storage
    for j in range(T):
        s[j,0] = (H[0,0]*sc[0,0] + H[0,1]*sc[0,1] + H[0,2]*sc[0,2])
        s[j,1] = (H[1,0]*sc[0,0] + H[1,1]*sc[0,1] + H[1,2]*sc[0,2] + eps[j,1])
        s[j,2] = (H[2,0]*sc[0,0] + H[2,1]*sc[0,1] + H[2,2]*sc[0,2] + eps[j,2])
        sc[0,0] = s[j,0]
        sc[0,1] = s[j,1]
        sc[0,2] = s[j,2]
    #GENERATE CONTROL VARIABLES
    R=np.zeros((T,1))
    C=np.zeros((T,1))
    prod=np.zeros((T,1))
    HH=np.zeros((T,1))
    price=np.zeros((T,1))
    Y=np.zeros((T,1))
    I=np.zeros((T,1))
    for j in range(T):
        prod[j,0] = (G[0,0]*s[j,0] + G[0,1]*s[j,1] + G[0,2]*s[j,2])
        R[j,0] = (G[1,0]*s[j,0] + G[1,1]*s[j,1] + G[1,2]*s[j,2])
        price[j,0] = (G[2,0]*s[j,0] + G[2,1]*s[j,1] + G[2,2]*s[j,2])
        C[j,0] = (G[3,0]*s[j,0] + G[3,1]*s[j,1] + G[3,2]*s[j,2])
        HH[j,0] = (G[4,0]*s[j,0] + G[4,1]*s[j,1] + G[4,2]*s[j,2])
        Y[j,0] = s[j,1] + (s[j,0]*theta) + (HH[j,0]*(1-theta))
        I[j,0] = (ybar/(ybar-cbar))*Y[j,0] - (ybar/(ybar-cbar))*C[j,0]
    #HP FILTER - we detrend the series
    hI, hI_trend = sm.tsa.filters.hpfilter(I,1600)
    hK, hK_trend = sm.tsa.filters.hpfilter(s[:,0],1600)
    hH, hH_trend = sm.tsa.filters.hpfilter(HH,1600)
    hprod, hprod_trend = sm.tsa.filters.hpfilter(prod,1600)
    hprice, hprice_trend = sm.tsa.filters.hpfilter(price, 1600)
    hY, hY_trend = sm.tsa.filters.hpfilter(Y,1600)
    hC, hC_trend = sm.tsa.filters.hpfilter(C,1600)
    #LET'S COMPUTE THE STANDARD DEVIATION
    sstd[i,0] = np.std(hY)
    sstd[i,1] = np.std(hC)
    sstd[i,2] = np.std(hI)
    sstd[i,3] = np.std(hK)
    sstd[i,4] = np.std(hH)
    sstd[i,5] = np.std(hprod)
    sstd[i,6] = np.std(hprice)
    #LET'S COMPUTE THE CORRELATION WITH OUTPUT
    scorr[i,0] = scipy.stats.spearmanr(hY,hY)[0]
    scorr[i,1] = scipy.stats.spearmanr(hC,hY)[0]
    scorr[i,2] = scipy.stats.spearmanr(hI,hY)[0]
    scorr[i,3] = scipy.stats.spearmanr(hK,hY)[0]
    scorr[i,4] = scipy.stats.spearmanr(hH,hY)[0]
    scorr[i,5] = scipy.stats.spearmanr(hprod,hY)[0]
    scorr[i,6] = scipy.stats.spearmanr(hprice,hY)[0]
    
#LET'S COMPUTE THE MEAN OF THE STANDARD ERRORS AND CORRELATION (AFTER THE MONTECARLO DRAWS)

stdY=(np.mean(sstd[:,0]))*100
stdC=(np.mean(sstd[:,1]))*100
stdI=(np.mean(sstd[:,2]))*100
stdK=(np.mean(sstd[:,3]))*100
stdH=(np.mean(sstd[:,4]))*100
stdprod=(np.mean(sstd[:,5]))*100
stdprice=(np.mean(sstd[:,6]))*100

corrY=np.mean(scorr[:,0])
corrC=np.mean(scorr[:,1])
corrI=np.mean(scorr[:,2])
corrK=np.mean(scorr[:,3])
corrH=np.mean(scorr[:,4])
corrprod=np.mean(scorr[:,5])
corrprice=np.mean(scorr[:,6])

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.58it/s]


In [7]:
print ('The standard deviation with without shock to money growth:')   
print ('Output: ', stdY)
print ('Consumption: ', stdC)
print ('Investment: ',stdI)
print ('Capital: ',stdK)
print ('Hours: ', stdH)
print ('Productivity: ', stdprod)
print ('Price: ', stdprice)
print ('While the correlation with output is:')
print ('Output: ', corrY)
print ('Consumption: ', corrC)
print ('Investment: ',corrI)
print ('Capital: ',corrK)
print ('Hours: ', corrH)
print ('Productivity: ', corrprod)
print ('Price: ', corrprice)

The standard deviation with without shock to money growth:
Output:  1.7683234319154961
Consumption:  0.5096601064444285
Investment:  5.24911545120429
Capital:  0.4756570621666353
Hours:  1.3458883231773568
Productivity:  0.5090695437588074
Price:  0.509660106444429
While the correlation with output is:
Output:  1.0
Consumption:  0.8623619584944371
Investment:  0.9788268208001264
Capital:  0.05570534995660065
Hours:  0.9789690049711988
Productivity:  0.8630222914858362
Price:  -0.8623619584944371


##### We simulate again the economy but this time imposing a standard deviation greater than 0.

In [8]:
#LET'S COMPUTE THE MONTECARLO DRAWS - SHOCK TO MONEY GROWTH

sigma2=0.009 #we impose the shock to money growth equal to zero

for i in tqdm(range(mc_reps)):
    eps = np.zeros((T,m))  #where we magazine the state variables
    shock1 = np.random.normal(mu1,sigma1,T)
    shock2 = np.random.normal(mu2,sigma2,T) 
    eps[:,1]=shock1
    eps[:,2]=shock2
    #GENERATE THE STATE VARIABLES
    s=np.zeros((T,m))
    sc=np.zeros((1,3)) #temporary storage
    for j in range(T):
        s[j,1] = (H[1,0]*sc[0,0] + H[1,1]*sc[0,1] + H[1,2]*sc[0,2] + eps[j,1])
        s[j,2] = (H[2,0]*sc[0,0] + H[2,1]*sc[0,1] + H[2,2]*sc[0,2] + eps[j,2])
        s[j,0] = (H[0,0]*sc[0,0] + H[0,1]*sc[0,1] + H[0,2]*sc[0,2])
        sc[0,0] = s[j,0]
        sc[0,1] = s[j,1]
        sc[0,2] = s[j,2]
    #GENERATE CONTROL VARIABLES
    R=np.zeros((T,1))
    C=np.zeros((T,1))
    prod=np.zeros((T,1))
    HH=np.zeros((T,1))
    price=np.zeros((T,1))
    Y=np.zeros((T,1))
    I=np.zeros((T,1))
    for j in range(T):
        prod[j,0] = (G[0,0]*s[j,0] + G[0,1]*s[j,1] + G[0,2]*s[j,2])
        R[j,0] = (G[1,0]*s[j,0] + G[1,1]*s[j,1] + G[1,2]*s[j,2])
        price[j,0] = (G[2,0]*s[j,0] + G[2,1]*s[j,1] + G[2,2]*s[j,2])
        C[j,0] = (G[3,0]*s[j,0] + G[3,1]*s[j,1] + G[3,2]*s[j,2])
        HH[j,0] = (G[4,0]*s[j,0] + G[4,1]*s[j,1] + G[4,2]*s[j,2])
        Y[j,0] = s[j,1] + (s[j,0]*theta) + (HH[j,0]*(1-theta))
        I[j,0] = (ybar/(ybar-cbar))*Y[j,0] - (ybar/(ybar-cbar))*C[j,0]
    #HP FILTER
    hI, hI_trend = sm.tsa.filters.hpfilter(I,1600)
    hK, hK_trend = sm.tsa.filters.hpfilter(s[:,0],1600)
    hH, hH_trend = sm.tsa.filters.hpfilter(HH,1600)
    hprod, hprod_trend = sm.tsa.filters.hpfilter(prod,1600)
    hprice, hprice_trend = sm.tsa.filters.hpfilter(price, 1600)
    hY, hY_trend = sm.tsa.filters.hpfilter(Y,1600)
    hC, hC_trend = sm.tsa.filters.hpfilter(C,1600)
    #LET'S COMPUTE THE STANDARD DEVIATION
    #simy = np.hstack((Y,C,I,s[:,0],HH,prod,price))
    sstd[i,0] = np.std(hY)
    sstd[i,1] = np.std(hC)
    sstd[i,2] = np.std(hI)
    sstd[i,3] = np.std(hK)
    sstd[i,4] = np.std(hH)
    sstd[i,5] = np.std(hprod)
    sstd[i,6] = np.std(hprice)
    #LET'S COMPUTE THE CORRELATION WITH OUTPUT
    scorr[i,0] = scipy.stats.spearmanr(hY,hY)[0]
    scorr[i,1] = scipy.stats.spearmanr(hC,hY)[0]
    scorr[i,2] = scipy.stats.spearmanr(hI,hY)[0]
    scorr[i,3] = scipy.stats.spearmanr(hK,hY)[0]
    scorr[i,4] = scipy.stats.spearmanr(hH,hY)[0]
    scorr[i,5] = scipy.stats.spearmanr(hprod,hY)[0]
    scorr[i,6] = scipy.stats.spearmanr(hprice,hY)[0]

stdY2=(np.mean(sstd[:,0]))*100
stdC2=(np.mean(sstd[:,1]))*100
stdI2=(np.mean(sstd[:,2]))*100
stdK2=(np.mean(sstd[:,3]))*100
stdH2=(np.mean(sstd[:,4]))*100
stdprod2=(np.mean(sstd[:,5]))*100
stdprice2=(np.mean(sstd[:,6]))*100

corrY2=np.mean(scorr[:,0])
corrC2=np.mean(scorr[:,1])
corrI2=np.mean(scorr[:,2])
corrK2=np.mean(scorr[:,3])
corrH2=np.mean(scorr[:,4])
corrprod2=np.mean(scorr[:,5])
corrprice2=np.mean(scorr[:,6])

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 106.05it/s]


In [9]:
print ('The standard deviation with shock to money growth and g=1.015 is:')   
print ('Output: ', stdY2)
print ('Consumption: ', stdC2)
print ('Investment: ',stdI2)
print ('Capital: ',stdK2)
print ('Hours: ', stdH2)
print ('Productivity: ', stdprod2)
print ('Price: ', stdprice2)
print ('While the correlation with output is:')
print ('Output: ', corrY2)
print ('Consumption: ', corrC2)
print ('Investment: ',corrI2)
print ('Capital: ',corrK2)
print ('Hours: ', corrH2)
print ('Productivity: ', corrprod2)
print ('Price: ', corrprice2)

The standard deviation with shock to money growth and g=1.015 is:
Output:  1.766027630413555
Consumption:  0.6562209193647036
Investment:  5.42793252636509
Capital:  0.47777236563298564
Hours:  1.3464082244486257
Productivity:  0.5094670779765396
Price:  0.6562209193647038
While the correlation with output is:
Output:  1.0
Consumption:  0.6722968357926301
Investment:  0.9308136826323681
Capital:  0.05533201294089798
Hours:  0.9783565138483391
Productivity:  0.8579242484021148
Price:  -0.6722968357926301


##### Finally, we simulate one more time the economy, but this time with an higher average growth rate of money. So, we need to compute again also the policy and transition functions.

In [10]:
# NOW WE WANT TO RE-RUN THE MODEL BUT WITH A DIFFERENT LEVEL OF MONEY GROWTH g=1.15

gbar=1.15
rbar=(1/beta)-(1-delta)
wbar=(1-theta)*((rbar/theta)**(theta/(theta-1)))
cbar=-((beta*wbar)/(gbar*matrixB))
pbar=(1/cbar)
kbar=cbar/((rbar/theta)-delta)
hbar=((rbar/theta)**(1/(1-theta)))*kbar
ybar=cbar+delta*kbar
#NUMBER OF VARIABLES
nvar=8
nk=3
#LET'S EXPRESS THE MODEL AS Aw(t+1)=Bw(t)
A=np.asmatrix(np.zeros((8,8)))
B=np.asmatrix(np.zeros((8,8)))
#LET'S COSTRUCT MATRIX A
A[0,3]=1
A[0,4]=-beta*rbar
A[1,2]=1
A[1,5]=1
A[1,6]=1
A[3,0]=kbar
A[6,1]=1
A[7,2]=1
#LET'S CONSTRUCT MATRIX B
B[0,3]=1
B[1,3]=1
B[1,5]=1
B[2,5]=1
B[2,6]=1
B[3,0]=(1-delta+rbar)*kbar
B[3,3]=wbar*hbar
B[3,4]=rbar*kbar
B[3,5]=1/pbar
B[3,7]=wbar*hbar
B[4,0]=theta
B[4,1]=1
B[4,3]=-1
B[4,7]=-theta
B[5,0]=theta-1
B[5,1]=1
B[5,4]=-1
B[5,7]=-(theta-1)
B[6,1]=gamma
B[7,2]=alpha

#COMPUTE POLICY AND TRANSITION FUNCTIONS
G, H = compute_policy_transition(A, B, nk, nvar)

print("Transition Function in the Hansen Model")
print (H)
print("Policy Function in the Hansen Model")
print (G)

Generalized decomposition satisfies stability property
Unique solution
Transition Function in the Hansen Model
[[ 9.41816660e-01  1.55228314e-01  2.71253428e-02]
 [ 0.00000000e+00  9.50000000e-01 -2.67622768e-17]
 [ 0.00000000e+00  0.00000000e+00  4.80000000e-01]]
Policy Function in the Hansen Model
[[ 0.53158781  0.4702745   0.03122343]
 [-0.94504499  1.94173422 -0.05550833]
 [-0.53158781 -0.4702745   0.44877657]
 [ 0.53158781  0.4702745  -0.44877657]
 [-0.4766328   1.47145973 -0.08673176]]


C:\Users\matti\AppData\Local\Temp/ipykernel_16660/305111649.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  geEigenvalues[ii]=np.real(BB[ii,ii]/AA[ii,ii])


In [11]:
#LET'S SETUP THE MONTECARLO PARAMETERS
mc_reps=1000
var=10
sstd=np.zeros((mc_reps,var))
scorr=np.zeros((mc_reps,var))
sigma1=0.00721 #shock to productivity
sigma2=0.009 #shock to money
mu1=(math.log(1))*(1-gamma) #mean of the two shocks
mu2=(math.log(1.15))*(1-alpha)
T=115 #number of quarters

In [12]:
#LET'S COMPUTE THE MONTECARLO DRAWS - SHOCK TO MONEY GROWTH AND HIGHER AVERAGE
for i in tqdm(range(mc_reps)):
    m=3  #state variables
    eps = np.zeros((T,m))  #where we magazine the state variables
    shock1 = np.random.normal(mu1,sigma1,T)
    shock2 = np.random.normal(mu2,sigma2,T)
    eps[:,1]=shock1
    eps[:,2]=shock2
    #GENERATE THE STATE VARIABLES
    s=np.zeros((T,m))
    sc=np.zeros((1,3)) #temporary storage
    for j in range(T):
        s[j,1] = (H[1,0]*sc[0,0] + H[1,1]*sc[0,1] + H[1,2]*sc[0,2] + eps[j,1])
        s[j,2] = (H[2,0]*sc[0,0] + H[2,1]*sc[0,1] + H[2,2]*sc[0,2] + eps[j,2])
        s[j,0] = (H[0,0]*sc[0,0] + H[0,1]*sc[0,1] + H[0,2]*sc[0,2])
        sc[0,0] = s[j,0]
        sc[0,1] = s[j,1]
        sc[0,2] = s[j,2]
    #GENERATE CONTROL VARIABLES
    R=np.zeros((T,1))
    C=np.zeros((T,1))
    prod=np.zeros((T,1))
    HH=np.zeros((T,1))
    price=np.zeros((T,1))
    Y=np.zeros((T,1))
    I=np.zeros((T,1))
    for j in range(T):
        prod[j,0] = (G[0,0]*s[j,0] + G[0,1]*s[j,1] + G[0,2]*s[j,2])
        R[j,0] = (G[1,0]*s[j,0] + G[1,1]*s[j,1] + G[1,2]*s[j,2])
        price[j,0] = (G[2,0]*s[j,0] + G[2,1]*s[j,1] + G[2,2]*s[j,2])
        C[j,0] = (G[3,0]*s[j,0] + G[3,1]*s[j,1] + G[3,2]*s[j,2])
        HH[j,0] = (G[4,0]*s[j,0] + G[4,1]*s[j,1] + G[4,2]*s[j,2])
        Y[j,0] = s[j,1] + (s[j,0]*theta) + (HH[j,0]*(1-theta))
        I[j,0] = (ybar/(ybar-cbar))*Y[j,0] - (ybar/(ybar-cbar))*C[j,0]
    #HP FILTER
    hI, hI_trend = sm.tsa.filters.hpfilter(I,1600)
    hK, hK_trend = sm.tsa.filters.hpfilter(s[:,0],1600)
    hH, hH_trend = sm.tsa.filters.hpfilter(HH,1600)
    hprod, hprod_trend = sm.tsa.filters.hpfilter(prod,1600)
    hprice, hprice_trend = sm.tsa.filters.hpfilter(price, 1600)
    hY, hY_trend = sm.tsa.filters.hpfilter(Y,1600)
    hC, hC_trend = sm.tsa.filters.hpfilter(C,1600)
    #LET'S COMPUTE THE STANDARD DEVIATION
    #simy = np.hstack((Y,C,I,s[:,0],HH,prod,price))
    sstd[i,0] = np.std(hY)
    sstd[i,1] = np.std(hC)
    sstd[i,2] = np.std(hI)
    sstd[i,3] = np.std(hK)
    sstd[i,4] = np.std(hH)
    sstd[i,5] = np.std(hprod)
    sstd[i,6] = np.std(hprice)
    #LET'S COMPUTE THE CORRELATION WITH OUTPUT
    scorr[i,0] = scipy.stats.spearmanr(hY,hY)[0]
    scorr[i,1] = scipy.stats.spearmanr(hC,hY)[0]
    scorr[i,2] = scipy.stats.spearmanr(hI,hY)[0]
    scorr[i,3] = scipy.stats.spearmanr(hK,hY)[0]
    scorr[i,4] = scipy.stats.spearmanr(hH,hY)[0]
    scorr[i,5] = scipy.stats.spearmanr(hprod,hY)[0]
    scorr[i,6] = scipy.stats.spearmanr(hprice,hY)[0]

#LET'S COMPUTE THE MEAN OF THE STANDARD ERRORS AND CORRELATION (AFTER THE MONTECARLO DRAWS)

stdY3=(np.mean(sstd[:,0]))*100
stdC3=(np.mean(sstd[:,1]))*100
stdI3=(np.mean(sstd[:,2]))*100
stdK3=(np.mean(sstd[:,3]))*100
stdH3=(np.mean(sstd[:,4]))*100
stdprod3=(np.mean(sstd[:,5]))*100
stdprice3=(np.mean(sstd[:,6]))*100

corrY3=np.mean(scorr[:,0])
corrC3=np.mean(scorr[:,1])
corrI3=np.mean(scorr[:,2])
corrK3=np.mean(scorr[:,3])
corrH3=np.mean(scorr[:,4])
corrprod3=np.mean(scorr[:,5])
corrprice3=np.mean(scorr[:,6])

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 106.44it/s]


In [13]:
print ('The standard deviation with g=1.15 is:')   
print ('Output: ', stdY3)
print ('Consumption: ', stdC3)
print ('Investment: ',stdI3)
print ('Capital: ',stdK3)
print ('Hours: ', stdH3)
print ('Productivity: ', stdprod3)
print ('Price: ', stdprice3)
print ('While the correlation with output is:')
print ('Output: ', corrY3)
print ('Consumption: ', corrC3)
print ('Investment: ',corrI3)
print ('Capital: ',corrK3)
print ('Hours: ', corrH3)
print ('Productivity: ', corrprod3)
print ('Price: ', corrprice3)

The standard deviation with g=1.15 is:
Output:  1.7764454515297932
Consumption:  0.6863822127548622
Investment:  5.507021440511763
Capital:  0.4819108608652446
Hours:  1.3560818224423705
Productivity:  0.512512206606107
Price:  0.6863822127548606
While the correlation with output is:
Output:  1.0
Consumption:  0.6552691864594019
Investment:  0.9226175175570109
Capital:  0.049134569557326604
Hours:  0.9778319577053579
Productivity:  0.854293687366843
Price:  -0.6552691864594019


In [14]:
headers= ['Costant money growth', 'Money growth, g=1.015 (with shocks)', 'Money growth, g=1.15 (with shocks)']
mydata = [ ('STANDARD DEVIATION','STANDARD DEVIATION','STANDARD DEVIATION'),
          (stdY, stdY2,stdY3),
          (stdC, stdC2, stdC3),
          (stdI, stdI2, stdI3),
          (stdK, stdK2, stdK3),
          (stdH, stdH2, stdH3),
          (stdprod, stdprod2, stdprod3),
          (stdprice, stdprice2, stdprice3),
          ('','',''),
          ('CORRELATION WITH OUTPUT','CORRELATION WITH OUTPUT','CORRELATION WITH OUTPUT'),
          (corrY, corrY2, corrY3),
          (corrC, corrC2, corrC3),
          (corrI, corrI2, corrI3),
          (corrK, corrK2, corrK3),
          (corrH, corrH2, corrH3),
          (corrprod, corrprod2, corrprod3),
          (corrprice, corrprice2, corrprice3)]
print(tabulate(mydata,headers=headers))

Costant money growth     Money growth, g=1.015 (with shocks)    Money growth, g=1.15 (with shocks)
-----------------------  -------------------------------------  ------------------------------------
STANDARD DEVIATION       STANDARD DEVIATION                     STANDARD DEVIATION
1.7683234319154961       1.766027630413555                      1.7764454515297932
0.5096601064444285       0.6562209193647036                     0.6863822127548622
5.24911545120429         5.42793252636509                       5.507021440511763
0.4756570621666353       0.47777236563298564                    0.4819108608652446
1.3458883231773568       1.3464082244486257                     1.3560818224423705
0.5090695437588074       0.5094670779765396                     0.512512206606107
0.509660106444429        0.6562209193647038                     0.6863822127548606

CORRELATION WITH OUTPUT  CORRELATION WITH OUTPUT                CORRELATION WITH OUTPUT
1.0                      1.0                     